# ReAct Agent

In [30]:
%pip install python-dotenv

  Obtaining dependency information for python-dotenv from https://files.pythonhosted.org/packages/6a/3e/b68c118422ec867fa7ab88444e1274aa40681c606d59ac27de5a5588f082/python_dotenv-1.0.1-py3-none-any.whl.metadata
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [32]:
%pip install openai

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/55/4c/906b5b32c4c01402ac3b4c3fc28f601443ac5c6f13c84a95dd178c8d545d/openai-1.52.2-py3-none-any.whl.metadata
  Using cached openai-1.52.2-py3-none-any.whl.metadata (24 kB)
  Obtaining dependency information for anyio<5,>=3.5.0 from https://files.pythonhosted.org/packages/e4/f5/f2b75d2fc6f1a260f340f0e7c6a060f4dd2961cc16884ed851b0d18da06a/anyio-4.6.2.post1-py3-none-any.whl.metadata
  Using cached anyio-4.6.2.post1-py3-none-any.whl.metadata (4.7 kB)
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/56/95/9377bcb415797e44274b51d46e3249eba641711cf3348050f76ee7b15ffc/httpx-0.27.2-py3-none-

In [33]:
import openai
import re
import httpx
import os

In [34]:
from openai import OpenAI
client = OpenAI()

MODEL = 'gpt-4o-mini'
prompt = 'Write a funny Chuck Norris joke!'

chat_completion = client.chat.completions.create(
  model=MODEL,
  messages=[{'role': 'user', 'content': prompt}]
)

In [35]:
chat_completion.choices[0].message.content 

'When Chuck Norris enters a room, he doesn’t turn the lights on. He turns the dark off!'

## Creating Agent Class

In [36]:
class Agent:
  def __init__(self, system=''):
    self.system = system
    self.messages = []
    
    if self.system:
      self.messages.append({'role': 'system', 'content': system})
      
  def __call__(self, prompt):
    self.messages.append({'role': 'user', 'content': prompt})
    result = self.execute()
    self.messages.append({'role': 'assistant', 'content': result})
    return result
  
  def execute(self, model=MODEL, temperature=0):
    completion = client.chat.completions.create(
      model=model, 
      temperature=temperature,
      messages=self.messages
    )
    
    return completion.choices[0].message.content

## Create the ReAct Prompt

In [37]:
prompt = '''
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_cost:
e.g. get_cost: book
returns the cost of a book

wikipedia:
e.g. wikipedia: LangChain
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session #1:

Question: How much does a pen cost?
Thought: I should look the pen cost using get_cost
Action: get_cost: pen
PAUSE

You will be called again with this:

Observation: A pen costs $5

You then output:

Answer: A pen costs $5


Example session #2

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
'''.strip()

## Create Tools

In [38]:
%pip install httpx


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [39]:
#1 calculate() function 
def calculate(inputs):
  """
  Calculate function takes a string, evaluates, and returns the result
  """
  return eval(inputs)

#2 get_cost()
def get_cost(item):
  if item in 'pen':
    return('A pen costs $5')
  elif item in 'book':
    return('A book costs $20')
  elif item in 'stapler':
    return('A stapler costs $10')
  else:
    return('A random item for writing costs $12')

#3 wikipedia()
def wikipedia(q):
  response = httpx.get('https://en.wikipedia.org/w/api.php', params={
    'action': 'query',
    'list': 'search',
    'srsearch': q,
    'format': 'json'
  })
  results = response.json().get('query').get('search')
  
  if not results:
    return None
  
  return results[0]['snippet']
  

In [40]:
wikipedia('langchain')

'<span class="searchmatch">LangChain</span> is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration'

In [41]:
known_actions = {
  'calculate': calculate,
  'get_cost': get_cost,
  'wikipedia': wikipedia
}

## Testing the Agent

In [42]:
agent = Agent(prompt)

In [44]:
result = agent('how much does a pen cost?')
print(result)

Thought: I should look up the cost of a pen using get_cost. 
Action: get_cost: pen 
PAUSE


In [49]:
next_prompt = f"Observation: {get_cost('pen')}"


In [50]:
agent(next_prompt)

'Answer: A pen costs $5.'

In [51]:
agent.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_cost:\ne.g. get_cost: book\nreturns the cost of a book\n\nwikipedia:\ne.g. wikipedia: LangChain\nReturns a summary from searching Wikipedia\n\nAlways look things up on Wikipedia if you have the opportunity to do so.\n\nExample session #1:\n\nQuestion: How much does a pen cost?\nThought: I should look the pen cost using get_cost\nAction: get_cost: pen\nPAUSE\n\nYou will be called again with this:\n\nObservation: A pen costs $5\n\nYou then output:\n\nAnswer: 

In [52]:
abot = Agent(prompt)

In [53]:
question = '''how does a pen and a book cost in total?'''
abot(question)

'Thought: I need to find the cost of both a pen and a book to calculate the total cost. I will first look up the cost of a pen and then the cost of a book. \nAction: get_cost: pen\nPAUSE'